In [44]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [45]:
# Function to classify a single pixel based on RGB thresholds
def classify_pixel(r, g, b):
    if b > r and b > g:  # Predominantly blue
        return "Highest_moisture"
    elif g > r and g > b:  # Predominantly green
        return "Highest_moisture"
    elif r > g and r > b:  # Predominantly red
        return "Dry"
    elif r > b and g > b:  # Mix of red and green (orange/yellow)
        return "Moderate_moisture"
    else:  # Default
        return "Moderate_moisture"

In [46]:
# Function to classify an image based on majority pixel class
def classify_image(image):
    classes = {"Highest_moisture": 0, "Moderate_moisture": 0, "Dry": 0}
    for row in image:
        print(row)
        for pixel in row:
            r, g, b = pixel
            class_label = classify_pixel(r, g, b)
            classes[class_label] += 1
    # Return the majority class
    return max(classes, key=classes.get)

In [47]:
data = tf.keras.preprocessing.image_dataset_from_directory(
    f"../data/Soil_classification_dataset_preprocessed",
    image_size=(96, 96),
    color_mode='rgb',
    batch_size=1,
)
classes = dict(enumerate(data.class_names))
reverse_classes = {v: k for k, v in classes.items()}
images = data.as_numpy_iterator()

Found 15002 files belonging to 3 classes.


In [ ]:
# Classify images using the RGB classifier
labels = []
predictions = []
for image in images:
    X, y_true = image
    labels.extend(y_true)
    predictions.append(classify_image(X[0]))
predictions = map(lambda x: reverse_classes[x], predictions)


In [40]:
labels = map(lambda x: classes[x], labels)
reverse_classes = {v: k for k, v in classes.items()}
predictions = map(lambda x: reverse_classes[x], predictions)
labels = map(lambda x: reverse_classes[x], labels)

In [43]:
[*labels]

KeyError: 'Moderate_moisture'

In [41]:
# Evaluate the model
accuracy = accuracy_score(labels, predictions)
report = classification_report(labels, predictions)
conf_matrix = confusion_matrix(labels, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

InvalidParameterError: The 'y_true' parameter of accuracy_score must be an array-like or a sparse matrix. Got <map object at 0x0000027050E90D90> instead.